# 1. 패키지 로드 및 폰트 설정

In [48]:
#!pip install pycaret

In [49]:
#!pip install holidays

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import holidays
import os

import matplotlib as mpl 
import matplotlib.pyplot as plt 
import matplotlib.font_manager as fm  
from pycaret.regression import *

# 2. Path 설정 및 데이터 로드


input_path에는 데이터가 위치한 폴더로 입력해주시면 되고 output_path에는 결과를  출력할 폴더 입력해주시면 됩니다.


In [45]:
input_path = './Dacon'
output_path = './Dacon'

In [46]:
df_train = pd.read_csv(input_path+'/train.csv')
df_test = pd.read_csv(input_path+'/test.csv')
df_submission = pd.read_csv(input_path+'/sample_submission.csv')
weather1 = pd.read_csv(input_path+'./충무공동_강수형태_201602_201701.csv')
weather2 = pd.read_csv(input_path+'./충무공동_강수형태_201702_201801.csv')
weather3 = pd.read_csv(input_path+'./충무공동_강수형태_201802_201901.csv')
weather4 = pd.read_csv(input_path+'./충무공동_강수형태_201902_202001.csv')
weather5 = pd.read_csv(input_path+'./충무공동_강수형태_202002_202101.csv')
weather6 = pd.read_csv(input_path+'./충무공동_강수형태_202102_202104.csv')

## 2-1. 강수량 데이터

In [16]:
# 컬럼명 변경
csv_list = [weather1, weather2, weather3, weather4, weather5, weather6]
for csv in csv_list:
    csv.columns = ['일자','hour','type']

In [17]:
weather = pd.concat([weather1,weather2,weather3,weather4,weather5,weather6],ignore_index=True)
weather.head(4)

,일자,hour,type
0,1,0.0,0.0
1,1,100.0,0.0
2,1,200.0,0.0
3,1,300.0,0.0


In [18]:
# weather_use => 필요한 시간대의 날씨 정보만 포함
tmp = weather[weather['hour']>=200] # 200 이상으로 필터링
tmp2 = tmp[tmp['hour']<=500] # 500 이하로 필터링
weather_use = tmp2.copy() # 오류 방지를 위해 복사본 저장
weather_use = weather_use.reset_index(drop=True) # 인덱스 초기화
weather_use.head(4)

,일자,hour,type
0,1,200.0,0.0
1,1,300.0,0.0
2,1,400.0,0.0
3,1,500.0,0.0


In [19]:
start_date = pd.to_datetime('2016-02-01')
end_date = pd.to_datetime('2021-04-30')
start_date

Timestamp('2016-02-01 00:00:00')

In [20]:
# 시작일자와 종료 일자를 받아 리스트에 날짜 당 N개의 원소를 입력하는 함수
def make_day_list(start, end, n=1):
    tmp_list = []
    while start.strftime('%Y%m%d') != (end+ timedelta(days=1)).strftime('%Y%m%d'):
        for i in range(n):
            tmp_list.append(start.strftime('%Y-%m-%d'))
        start += timedelta(days=1) 
    return tmp_list

In [21]:
date_list = make_day_list(start_date,end_date,4)

weather_use.loc[:,'일자'] = date_list
weather_use.head(4)

,일자,hour,type
0,2016-02-01,200.0,0.0
1,2016-02-01,300.0,0.0
2,2016-02-01,400.0,0.0
3,2016-02-01,500.0,0.0


In [22]:
# 해당 시간에 1시간이라도 비가 오지 않은 날 list 추출 및 df 생성
no_rainy_day = list(weather_use[(weather_use.loc[:,'type'] == -1) |weather_use.loc[:,'type'] == 0]['일자'].unique())
tmp_df = pd.DataFrame(weather_use['일자'].unique(),columns=['일자'])
tmp_df.head(4)

,일자
0,2016-02-01
1,2016-02-02
2,2016-02-03
3,2016-02-04


In [23]:
# 전체 기간에서 비가 온날은 1 비가 오지 않은 날은 0을 생성하는 list
rainy_list = []
for i in range(len(tmp_df)):
    if tmp_df['일자'][i] in no_rainy_day:
        rainy_list.append(0)
    else:
        rainy_list.append(1)

In [24]:
tmp_df['점심강수여부'] = rainy_list
tmp_df.head(4)

,일자,점심강수여부
0,2016-02-01,0
1,2016-02-02,0
2,2016-02-03,0
3,2016-02-04,0


In [25]:
# train, test 데이터를 기준으로 병합
df_train = df_train.merge(tmp_df,how='left')
df_test = df_test.merge(tmp_df,how='left')
df_train.head(4)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,점심강수여부
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,0


## 2-2. 공휴일 데이터

공휴일 전날, 다음날 리스트 생성하여 관련성 파악

공휴일이 토/일인 경우는 리스트에서 제외


In [26]:
# 리스트를 입력 받아 +- 1의 날짜를 출력하는 함수
def holiday_close(day_list):
    close_list = []
    for day in holiday_list:
        if day.weekday() != 5 and day.weekday() != 6: # 토, 일요일이 아닌 경우에 리스트에 추가
            close_list.append((day - timedelta(days=1)).strftime('%Y-%m-%d'))
            close_list.append((day + timedelta(days=1)).strftime('%Y-%m-%d'))
    close_list = list(set(close_list)) # set을 통해 중복 제거 
    return sorted(close_list)  # 정렬된 리스트 반환

In [27]:
holiday_list = [] # 공휴일이 저장될 딕셔너리 변수
for i in range(2016,2022):
    for date, name in sorted(holidays.KR(years=i).items()):
        holiday_list.append(date)
close_holiday_list = holiday_close(holiday_list) # holiday_close 함수를 이용해 공휴일 전날/다음날 리스트 생성

In [28]:
# 공휴일 전후 임시 df 생성
tmp_df = pd.DataFrame(close_holiday_list,columns=['일자'])
tmp_df['공휴일전후'] = 1
tmp_df.head(4)

,일자,공휴일전후
0,2015-12-31,1
1,2016-01-02,1
2,2016-02-07,1
3,2016-02-08,1


In [29]:
# train / test에 merge
df_train = df_train.merge(tmp_df,how='left').fillna(0)
df_test = df_test.merge(tmp_df,how='left').fillna(0)
df_train.head(4)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,점심강수여부,공휴일전후
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,0,0.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,0,0.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,0,0.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,0,0.0


# 3. Feature Engineering

In [30]:
df_train[['현본사소속재택근무자수', '중식계', '석식계','점심강수여부','공휴일전후']] = df_train[['현본사소속재택근무자수', '중식계', '석식계','점심강수여부','공휴일전후']].astype('int')
df_test['현본사소속재택근무자수'] = df_test['현본사소속재택근무자수'].astype('int')

df_train['일자'] = pd.to_datetime(df_train['일자'])
df_test['일자'] = pd.to_datetime(df_test['일자'])

df_train['년'] = df_train['일자'].dt.year
df_train['월'] = df_train['일자'].dt.month
df_train['일'] = df_train['일자'].dt.day
df_train['주'] = df_train['일자'].dt.week
df_train['요일'] = df_train['일자'].dt.weekday
df_train['출근'] = df_train['본사정원수']-(df_train['본사휴가자수']+df_train['본사출장자수']+df_train['현본사소속재택근무자수'])
df_train['휴가비율'] = df_train['본사휴가자수']/df_train['본사정원수']
df_train['출장비율'] = df_train['본사출장자수']/df_train['본사정원수']
df_train['야근비율'] = df_train['본사시간외근무명령서승인건수']/df_train['출근']
df_train['재택비율'] = df_train['현본사소속재택근무자수']/df_train['본사정원수']
df_train['본사휴가자수log'] = np.log1p(df_train['본사휴가자수'])
df_train['본사시간외근무명령서승인건수log'] = np.log1p(df_train['본사시간외근무명령서승인건수'])

df_test['년'] = df_test['일자'].dt.year
df_test['월'] = df_test['일자'].dt.month
df_test['일'] = df_test['일자'].dt.day
df_test['주'] = df_test['일자'].dt.week
df_test['요일'] = df_test['일자'].dt.weekday
df_test['출근'] = df_test['본사정원수']-(df_test['본사휴가자수']+df_test['본사출장자수']+df_test['현본사소속재택근무자수'])
df_test['휴가비율'] = df_test['본사휴가자수']/df_test['본사정원수']
df_test['출장비율'] = df_test['본사출장자수']/df_test['본사정원수']
df_test['야근비율'] = df_test['본사시간외근무명령서승인건수']/df_test['출근']
df_test['재택비율'] = df_test['현본사소속재택근무자수']/df_test['본사정원수']
df_test['본사휴가자수log'] = np.log1p(df_test['본사휴가자수'])
df_test['본사시간외근무명령서승인건수log'] = np.log1p(df_test['본사시간외근무명령서승인건수'])

<ipython-input-30-72660828e089>:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_train['주'] = df_train['일자'].dt.week
<ipython-input-30-72660828e089>:23: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_test['주'] = df_test['일자'].dt.week


# 4. 메뉴 임베딩

## Train

점심 메뉴

In [31]:
# 일별 점심메뉴를 작은 리스트로 갖고 있는 큰 리스트 (lunch_train) 만들기 
lunch_train = []
for day in range(len(df_train)):
    tmp = df_train.iloc[day, 8].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    lunch_train.append(tmp) 


In [32]:
# lunch train data에 메뉴명별 칼럼 만들기 (밥, 국, 반찬1-3)
menu_len_list = []
bob = []; gook = []; banchan1 = []; banchan2 = []; banchan3 = []; kimchi = []; side = [];
for i, day_menu in enumerate(lunch_train):
    bob_tmp = day_menu[0]; bob.append(bob_tmp)
    gook_tmp = day_menu[1]; gook.append(gook_tmp)
    banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
    banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
    banchan3_tmp = day_menu[4]; banchan3.append(banchan3_tmp)

    if i < 1067:
        kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
        side_tmp = day_menu[-2]; side.append(side_tmp)
    else: 
        kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
        side_tmp  = day_menu[-1]; side.append(side_tmp)
    menu_len_list.append([len(day_menu),i])

In [33]:
train_ln = df_train.copy()

train_ln['밥'] = bob
train_ln['국'] = gook
train_ln['반찬1'] = banchan1; train_ln['반찬2'] = banchan2; train_ln['반찬3'] = banchan3
train_ln['김치'] = kimchi
train_ln['사이드'] = side

In [34]:
train_ln.head(5)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,재택비율,본사휴가자수log,본사시간외근무명령서승인건수log,밥,국,반찬1,반찬2,반찬3,김치,사이드
0,2016-02-01,0,2601,50,150,238,0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",...,0.0,3.931826,5.476464,쌀밥/잡곡밥,오징어찌개,쇠불고기,계란찜,청포묵무침,포기김치,요구르트
1,2016-02-02,1,2601,50,173,319,0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",...,0.0,3.931826,5.768321,쌀밥/잡곡밥,김치찌개,가자미튀김,모둠소세지구이,마늘쫑무침,배추겉절이,요구르트
2,2016-02-03,2,2601,56,180,111,0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",...,0.0,4.043051,4.718499,카레덮밥,팽이장국,치킨핑거,쫄면야채무침,견과류조림,포기김치,요구르트
3,2016-02-04,3,2601,104,220,355,0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",...,0.0,4.653960,5.874931,쌀밥/잡곡밥,쇠고기무국,주꾸미볶음,부추전,시금치나물,포기김치,요구르트
4,2016-02-05,4,2601,278,181,34,0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",...,0.0,5.631212,3.555348,쌀밥/잡곡밥,떡국,돈육씨앗강정,우엉잡채,청경채무침,포기김치,요구르트


저녁 메뉴

In [35]:
# 일별 저녁메뉴를 작은 리스트로 갖고 있는 큰 리스트 (dinner_train) 만들기 
dinner_train = []
for day in range(len(df_train)):
    tmp = df_train.iloc[day, 9].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    dinner_train.append(tmp) 


In [36]:
# dinner train data에 메뉴명별 칼럼 만들기 (밥, 국, 반찬1-3)
bob = []; gook = []; banchan1 = []; banchan2 = []; banchan3 = []; kimchi = []; side = []
for i, day_menu in enumerate(dinner_train):
    if (len(day_menu) < 4 ):
        bob.append('*')
        gook.append('*')
        banchan1.append('*')
        banchan2.append('*')
        banchan3.append('*')

        if i < 1067:
            kimchi.append('*')
            side.append('*')
        else: 
            kimchi.append('*')
            side.append('*')

    elif (len(day_menu)==4):
        bob_tmp = day_menu[0]; bob.append(bob_tmp)
        gook_tmp = day_menu[1]; gook.append(gook_tmp)
        banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
        banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
        banchan3.append('*')

        if i < 1067:
            kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
            side_tmp = day_menu[-2]; side.append(side_tmp)
        else: 
            kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
            side_tmp  = day_menu[-1]; side.append(side_tmp)
    
    else :
        bob_tmp = day_menu[0]; bob.append(bob_tmp)
        gook_tmp = day_menu[1]; gook.append(gook_tmp)
        banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
        banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
        banchan3_tmp = day_menu[4]; banchan3.append(banchan3_tmp)

        if i < 1067:
            kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
            side_tmp = day_menu[-2]; side.append(side_tmp)
        else: 
            kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
            side_tmp  = day_menu[-1]; side.append(side_tmp)
    

In [37]:
train_dn = df_train.copy()

train_dn['밥'] = bob
train_dn['국'] = gook
train_dn['반찬1'] = banchan1; train_dn['반찬2'] = banchan2; train_dn['반찬3'] = banchan3
train_dn['김치'] = kimchi
train_dn['사이드'] = side

## Test

점심 메뉴

In [38]:
# 일별 점심메뉴를 작은 리스트로 갖고 있는 큰 리스트 (lunch_test) 만들기 
lunch_test = []
for day in range(len(df_test)):
    tmp = df_test.iloc[day, 8].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    lunch_test.append(tmp) 


In [39]:
# lunch test data에 메뉴명별 칼럼 만들기 (밥, 국, 반찬1-3)
menu_len_list = []
bob = []; gook = []; banchan1 = []; banchan2 = []; banchan3 = []; kimchi = []; side = [];
for i, day_menu in enumerate(lunch_test):
    bob_tmp = day_menu[0]; bob.append(bob_tmp)
    gook_tmp = day_menu[1]; gook.append(gook_tmp)
    banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
    banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
    banchan3_tmp = day_menu[4]; banchan3.append(banchan3_tmp)

    if i < 1067:
        kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
        side_tmp = day_menu[-2]; side.append(side_tmp)
    else: 
        kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
        side_tmp  = day_menu[-1]; side.append(side_tmp)
    menu_len_list.append([len(day_menu),i])

In [40]:
test_ln = df_test.copy()


test_ln['밥'] = bob
test_ln['국'] = gook
test_ln['반찬1'] = banchan1; test_ln['반찬2'] = banchan2; test_ln['반찬3'] = banchan3
test_ln['김치'] = kimchi
test_ln['사이드'] = side

저녁 메뉴

In [41]:
# 일별 저녁메뉴를 작은 리스트로 갖고 있는 큰 리스트 (dinner_test) 만들기 
dinner_test = []
for day in range(len(df_test)):
    tmp = df_test.iloc[day, 9].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    dinner_test.append(tmp) 


In [42]:
# dinner test data에 메뉴명별 칼럼 만들기 (밥, 국, 반찬1-3)
bob = []; gook = []; banchan1 = []; banchan2 = []; banchan3 = []; kimchi = []; side = []
for i, day_menu in enumerate(dinner_test):
    if (len(day_menu) < 4 ):
        bob.append('*')
        gook.append('*')
        banchan1.append('*')
        banchan2.append('*')
        banchan3.append('*')

        if i < 1067:
            kimchi.append('*')
            side.append('*')
        else: 
            kimchi.append('*')
            side.append('*')

    elif (len(day_menu)==4):
        bob_tmp = day_menu[0]; bob.append(bob_tmp)
        gook_tmp = day_menu[1]; gook.append(gook_tmp)
        banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
        banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
        banchan3.append('*')

        if i < 1067:
            kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
            side_tmp = day_menu[-2]; side.append(side_tmp)
        else: 
            kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
            side_tmp  = day_menu[-1]; side.append(side_tmp)
    
    else :
        bob_tmp = day_menu[0]; bob.append(bob_tmp)
        gook_tmp = day_menu[1]; gook.append(gook_tmp)
        banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
        banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
        banchan3_tmp = day_menu[4]; banchan3.append(banchan3_tmp)

        if i < 1067:
            kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
            side_tmp = day_menu[-2]; side.append(side_tmp)
        else: 
            kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
            side_tmp  = day_menu[-1]; side.append(side_tmp)
    

In [43]:
test_dn = df_test.copy()

test_dn['밥'] = bob
test_dn['국'] = gook
test_dn['반찬1'] = banchan1; test_dn['반찬2'] = banchan2; test_dn['반찬3'] = banchan3
test_dn['김치'] = kimchi
test_dn['사이드'] = side

# 5. Pycaret

 ## 중식계

In [32]:
train_ln.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '점심강수여부', '공휴일전후',
       '년', '월', '일', '주', '출근', '휴가비율', '출장비율', '야근비율', '재택비율', '본사휴가자수log',
       '본사시간외근무명령서승인건수log', '밥', '국', '반찬1', '반찬2', '반찬3', '김치', '사이드'],
      dtype='object')

In [39]:
r_ln = setup(data=train_ln,target='중식계', session_id=6088,categorical_features=['점심강수여부','공휴일전후'],ignore_features=['재택비율','현본사소속재택근무자수','본사휴가자수','본사시간외근무명령서승인건수','조식메뉴','석식메뉴','석식계'])#,'공휴일전후'])

,Description,Value
0,session_id,6088
1,Target,중식계
2,Original Data,"(1205, 32)"
3,Missing Values,False
4,Numeric Features,10
5,Categorical Features,13
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(843, 1311)"


In [40]:
lunch_best_3 = compare_models(sort='MAE', n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,65.8766,7748.5126,87.5044,0.8224,0.1104,0.0810,7.1770
gbr,Gradient Boosting Regressor,71.3219,8973.7840,94.3282,0.7946,0.1175,0.0875,1.0710
xgboost,Extreme Gradient Boosting,72.7054,9261.4102,95.5057,0.7883,0.1202,0.0896,2.7870
lightgbm,Light Gradient Boosting Machine,72.8605,9218.0892,95.6703,0.7888,0.1211,0.0895,0.8370
rf,Random Forest Regressor,73.9686,9812.7970,98.6798,0.7759,0.1240,0.0909,1.5420
et,Extra Trees Regressor,74.7201,10430.8116,101.7778,0.7629,0.1279,0.0921,2.3260
lasso,Lasso Regression,79.9133,11614.3215,107.5148,0.7336,0.1346,0.0983,0.0650
br,Bayesian Ridge,82.4449,11955.1828,108.8413,0.7257,0.1371,0.1018,2.6490
omp,Orthogonal Matching Pursuit,83.4728,13046.3636,113.4311,0.7034,0.1439,0.1038,0.0820
ada,AdaBoost Regressor,84.9863,12041.8160,109.4143,0.7247,0.1356,0.1046,1.0390


In [41]:
#앙상블
blend_lunch = blend_models(estimator_list= lunch_best_3, optimize='MAE')
pred_holdout = predict_model(blend_lunch)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,71.9209,9429.2026,97.1041,0.7683,0.1105,0.0828
1,68.9051,7604.0389,87.2011,0.8278,0.0956,0.0768
2,71.4305,10994.0034,104.8523,0.7396,0.1474,0.0977
3,74.4001,9705.2519,98.5152,0.7692,0.1074,0.0825
4,64.1653,6950.6680,83.3707,0.8275,0.1059,0.0808
5,60.3804,6012.7065,77.5416,0.8469,0.1088,0.0775
6,63.4788,6109.4914,78.1632,0.8704,0.1001,0.0793
7,64.0857,6241.4062,79.0026,0.8445,0.0979,0.0778
8,65.4189,8035.4791,89.6408,0.8554,0.1235,0.0901
9,75.1157,10392.6733,101.9445,0.7863,0.1309,0.0909


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,61.9065,7191.4852,84.8026,0.8295,0.1044,0.0742


In [45]:
final_model = finalize_model(blend_lunch)

In [46]:
predictions = predict_model(final_model, data = test_ln)

In [47]:
df_submission['중식계'] = predictions['Label']

## 석식계

In [53]:
train_dn.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '점심강수여부', '공휴일전후',
       '년', '월', '일', '주', '출근', '휴가비율', '출장비율', '야근비율', '재택비율', '본사휴가자수log',
       '본사시간외근무명령서승인건수log', '밥', '국', '반찬1', '반찬2', '반찬3', '김치', '사이드'],
      dtype='object')

In [67]:
r_dn = setup(data=train_dn,target='석식계', session_id=2441,categorical_features=['점심강수여부','공휴일전후'],ignore_features=['재택비율','현본사소속재택근무자수','본사휴가자수','조식메뉴','중식메뉴','중식계','점심강수여부'])#,'공휴일전후'])

,Description,Value
0,session_id,2441
1,Target,석식계
2,Original Data,"(1205, 32)"
3,Missing Values,False
4,Numeric Features,11
5,Categorical Features,12
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(843, 1300)"


In [68]:
dinner_best_3 = compare_models(sort='MAE', n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,46.9467,4401.6176,65.9329,0.7690,0.4743,0.1125,1.4430
lightgbm,Light Gradient Boosting Machine,46.9913,4221.1860,64.2672,0.7786,0.5437,0.1127,0.7110
catboost,CatBoost Regressor,47.3378,4287.7991,65.0480,0.7753,0.6625,0.1113,6.5960
et,Extra Trees Regressor,47.6117,4757.3784,68.4275,0.7530,0.4374,0.1142,2.2850
xgboost,Extreme Gradient Boosting,47.8440,4534.5862,66.7820,0.7630,0.6021,0.1123,2.4640
gbr,Gradient Boosting Regressor,49.8501,4943.8713,69.8288,0.7397,0.6680,0.1186,0.9600
lasso,Lasso Regression,55.1814,5806.6672,75.7142,0.6922,0.7096,0.1329,0.0700
omp,Orthogonal Matching Pursuit,55.5644,6443.9269,79.7687,0.6583,0.6657,0.1327,0.0780
br,Bayesian Ridge,57.0712,6070.3460,77.4967,0.6764,0.7329,0.1351,2.3010
dt,Decision Tree Regressor,60.5018,7430.3119,85.8396,0.6056,0.3950,0.1427,0.0650


In [69]:
#앙상블
blend_dinner = blend_models(estimator_list= dinner_best_3, optimize='MAE')
pred_holdout = predict_model(blend_dinner)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,48.7391,4351.5285,65.9661,0.7318,0.5250,0.1165
1,48.4958,5508.6200,74.2201,0.7612,0.8651,0.1145
2,52.8491,5466.0513,73.9327,0.7487,0.8141,0.1367
3,40.6287,3191.3621,56.4921,0.7606,0.2219,0.0974
4,46.2712,4221.5404,64.9734,0.7947,0.6381,0.1119
5,37.8089,2488.1984,49.8818,0.8488,0.6281,0.0811
6,45.7851,4228.8871,65.0299,0.7806,0.5118,0.1020
7,37.7811,2522.6874,50.2264,0.8051,0.1519,0.0984
8,43.4265,3396.1766,58.2767,0.8575,0.6620,0.0920
9,50.1947,4689.3166,68.4786,0.8180,0.6758,0.1292


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,42.1016,3546.9327,59.5561,0.8142,0.5977,0.1000


In [72]:
final_model = finalize_model(blend_dinner)

In [73]:
predictions = predict_model(final_model, data = test_dn)

In [74]:
df_submission['석식계'] = predictions['Label']

In [75]:
df_submission

,일자,중식계,석식계
0,2021-01-27,1063.473957,381.983213
1,2021-01-28,1026.426714,488.513150
2,2021-01-29,638.367314,261.927738
3,2021-02-01,1304.745400,553.385245
4,2021-02-02,1127.046315,524.223223
5,2021-02-03,1073.586092,442.650976
6,2021-02-04,1041.730828,527.233265
7,2021-02-05,675.638020,411.330118
8,2021-02-08,1372.699021,650.415510
9,2021-02-09,1155.575070,556.985375


# 6. 결과 csv 저장

In [76]:
df_submission.to_csv('./submission_csv/submission.csv',index=False)